## Python libraries in this workshop

- [pandas](https://pandas.pydata.org/docs/getting_started/index.html)
- [geopandas](https://geopandas.org/en/stable/gallery/index.html)
- [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)


## Import libraries

In [2]:
# the regulars
import pandas as pd
import geopandas as gpd
import os
# earth engine
import ee
import json
from ee.ee_exception import EEException
from dotenv import load_dotenv
# Load the configuration variables from the .env file
load_dotenv()
# allow images to display in the notebook
from IPython.display import Image

In [3]:
private_key_file_path = os.getenv('GEE_SERVICE_ACCOUNT_PATH')

def geeAuthCredentials():
    #// todo - setup env for gee auth path to json service account file
    with open(private_key_file_path,  'r') as file:
        data = json.load(file)
    return data

## Authenticate Earth Engine

In [4]:
private_key_json = geeAuthCredentials()
service_account = private_key_json['client_email']

try:
    credentials = ee.ServiceAccountCredentials(service_account, private_key_file_path)
    ee.Initialize(credentials)
except EEException as e:
    print(str(e))

For this lab, we will use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [5]:
# coordinates of the Camp Fire
lat =  39.444012
lon = -121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-10-01'

# end date
end_date = '2019-01-31'

## Get Landsat 8 data

In [6]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)
            
            
# Load the Sentinel-2 image collection
sentinel = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [7]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

print('Total number:', sentinel.size().getInfo())

Total number: 8
Total number: 20


In [10]:
# information about the first image in our collection
landsat.first().getInfo()
sentinel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32610',
   'crs_transform': [60, 0, 499980, 0, -60, 4400040]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32610',
   'crs_transform': [10, 0, 499980, 0, -10, 4400040]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32610',
   'crs_transform': [10, 0, 499980, 0, -10, 4400040]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32610',
   'crs_transform': [10, 0, 499980, 0, -10, 4400040]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [11]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

0.05

In [12]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2018-10-07'

## Bands
<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/Landsat%208%20band%20designations.jpg" width=800>

[Source: USGS](https://www.usgs.gov/media/images/landsat-8-band-designations)


<img src="images/bands.jpg" width=1000>

[Source: Satellite Imaging Corporation](https://www.satimagingcorp.com/satellite-sensors/worldview-3/)

In [13]:
# what bands did we get?
landsat.first().bandNames().getInfo()
sentinel.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [14]:
# put the images in a list
landsat_list = landsat.toList(landsat.size())
sentinel_list = sentinel.toList(sentinel.size())

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine)

In [31]:
# set some parameters for the images
parameters = {
   'min': 7000,
   'max': 16000,
   'dimensions': 800, # square size in pixels
   'bands': ['B4', 'B3', 'B3'], # bands to display (r,g,b),
   'gamma': 1.0
}

In [32]:
def get_acquisition_time(image):
    # Get the image acquisition time metadata
    acquisition_time = ee.Date(image.get('system:time_start'))
    
    # Format the acquisition time as a string
    acquisition_time_str = acquisition_time.format('YYYY-MM-dd HH:mm:ss')
    
    return acquisition_time_str

In [35]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(sentinel.size().getInfo()):

    # when was this image taken?
    date = ee.Image(sentinel_list.get(11)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data 
    data.append(this_data)
    break
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 None Cloud cover: None


In [78]:


roi = ee.Geometry.Polygon(
        [[[33.438911421328484, -14.242571177249024],
          [33.438911421328484, -14.24864414767038],
          [33.44642160657018, -14.24864414767038],
          [33.44642160657018, -14.242571177249024]]], None, False)

image = ee.ImageCollection('COPERNICUS/S2_SR')\
.filterDate('2020-01-01', '2020-12-30')\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
.filterBounds(roi)\
.median()


# set some parameters for the images
parameters =  {
  # 'crs': 'EPSG:4326',
  'bands': ['B4', 'B3', 'B2'], 
  'min': 0,
  'max': 2500,
  # 'dimensions': 800, # square size in pixels
  # 'gamma': 1.0,
  # 'image': image.int16(),
  # 'description': 'Sentinel2AChennai',
  'scale': 10,
  # 'fileFormat': 'GeoTIFF',
  'region': roi,
  # 'maxPixels': 1e130,
}

print(ee.Image(image).getThumbUrl({
  # 'crs': 'EPSG:4326',
  'bands': ['B4', 'B3', 'B2'], 
  'min': 0,
  'max': 2500,
  # 'dimensions': 800, # square size in pixels
  # 'gamma': 1.0,
  # 'image': image.int16(),
  # 'description': 'Sentinel2AChennai',
  'scale': 10,
  # 'fileFormat': 'GeoTIFF',
  'region': roi,
  # 'maxPixels': 1e130,
}))

url = ee.Image(image).getDownloadURL({
  'crs': 'EPSG:4326',
  'bands': ['B4', 'B3', 'B2'], 
  'min': 7000,
  'max': 16000,
  'dimensions': 800, # square size in pixels
  'gamma': 1.1,
  'image': image.int16(),
  'single_image': True,
  # 'description': 'Sentinel2AChennai',
  # 'scale': 10,
  # 'fileFormat': 'GeoTIFF',
  'region': roi,
  # 'maxPixels': 1e13,
})
print(url)
display(Image(url = url))

# url = image.getDownloadURL({
#   'name': 'sentinel2_image',
#   'scale': 10,
#   'crs': 'EPSG:4326'
# })

# print('Download URL:', url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/39f29cf8598052eef37c300f165c7730-9f8e14974959ccef4ee51eb9a622b2e6:getPixels


EEException: Total request size (107143200 bytes) must be less than or equal to 50331648 bytes.

In [14]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,0,2018-10-07,0.05
1,1,2018-10-23,73.04
2,2,2018-11-08,11.83
3,3,2018-11-24,67.16
4,4,2018-12-10,56.09
5,5,2018-12-26,5.99
6,6,2019-01-11,80.06
7,7,2019-01-27,5.21


## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [15]:
# create a list of images we want (before, during, after)
landsat_sequence = [0,2,5]

In [19]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(2000) # meters

In [20]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [21]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 5 2018-12-26 Cloud cover: 5.99


<div class="alert alert-info">
Take a moment to inspect the three images above. What do they tell you? What do they NOT tell you?
</div>

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

<img src="images/ndvi.png" width=600>

- [Source: Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/)

### Calculating NDVI in Google Earth Engine

- https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [ ]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [ ]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

## Folium

Of course, we can't end the lab without an interactive map. For this, we can use [folium](https://python-visualization.github.io/folium/quickstart.html), which uses the open-source javascript mapping library [leaflet](https://leafletjs.com/).

- https://python-visualization.github.io/folium/quickstart.html

Google earth engine works with folium:
- https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

In [ ]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

In [ ]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Save the folium map as an html file

In [ ]:
my_map.save('camp.html')

## Evaluation

<img src="images/yoh.png">
A big thank you to all of you who have come today. Please take a moment to fill this survey, as we always want to hear from you.

https://docs.google.com/forms/d/e/1FAIpQLSdVTXFIt4-8c6NQGJc_qORMKIzEmvmBKWFTgU7Ek6AZEq3Xww/viewform


# Resources

- [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)
- [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
    